In [5]:
from autocorrect import spell
print ("****",spell("d"))




**** d


In [ ]:
## data reading both train and test

import pickle
data = open("C:\\Users\\63483\\Desktop\\EMO\\starterkitdata\\train.txt","r" , encoding="utf8").readlines()
structured_train_data=[]
for line in data[1:]:
    split_line=line.split("\t")
    structured_train_data.append([split_line[1],split_line[2],split_line[3],split_line[4]])
pickle.dump(structured_train_data,open("structured_train_data.p","wb"))   
print("data saved success")

In [1]:
## preprocess train data

import string
import pickle
import re
import unicodedata
import emoji
from re import finditer
from autocorrect import spell
import normalise
import unicodedata

def removeEmoticons(text):
    
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\u2600-\u26FF\u2700-\u27BF"                        
                           "]+", flags=re.UNICODE)
    
    preprocess_data=[]
    for sent in text:
        u=emoji_pattern.sub(r'', sent)
        preprocess_data.append(u)
    return preprocess_data


def removePunctuationAndConvertToLowercase(sentence):
    processed = list()
    #st=string.punctuation.replace("'", "")
    for line in sentence:
        for c in string.punctuation:
            line=line.replace(c,"")
        line=line.lower()
        if(len(line)>0):
            processed.append(line)
    return processed   

dictemp={}
emo_feature={}

def extract_emojis(str):
          temp = ''.join(c for c in str if c in emoji.UNICODE_EMOJI)         
          if(len(temp) > 0):
            return temp

           
def replaceEmoticons(sentence,mp):
    global emojistr
    processed = []
    sad_emoticons = {":-(", ":(", ":-|", ";-(", ";-<", "|-{" , "D‑':" , "D:<" , "D:", "D8" , "D;" , "D=" , "DX" , ":@" , ">:(",
                     ":{"  , ":‑c" , ":c" , ":'‑(" , ":'(" , ":‑###.." , ":###.." , "</3" , "<\3" 
                    }
    
    happy_emoticons = {":-)", ":)", ":o)", ":-}", ";-}", ":->", ";-)" , ":]" , ":>" , ":->" , "8-)", "8)" , ":o)" , ":c)" , 
                       ":^)" , "=]" , "=)" , ":‑D" , ":D" , "8‑D" , "8D" , "x‑D" , "xD" , "X‑D" , "XD" , "=D" , "=3" , "B^D" , 
                       ":-))" , ":')" , ":'‑)" , ":-*" , ":*" , ":×" , ":‑P" , ":P" , "x‑p" , "xp" , ":‑p" , ":p" , ":‑þ" , ":þ",
                       ":‑b", ":b" , "d:","=p" , ">:P" , "<3"  }
    
    angry_emoticons = { ":-||" , ">:[" , ">:(" , ":@" , ">:‑)" , ">:)" , "}:‑)" , "3:‑)" , ">;)" , "}:)" , "3:)"}
    
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\u2600-\u26FF\u2700-\u27BF"                       
                           "]+", flags=re.UNICODE)
    for line in sentence:
        vec=[0,0,0]
        words = list(line.split())
        for i in range(len(words)):
            curr_word=words[i]
            if(curr_word in happy_emoticons):
                #words[i]="happy"
                vec[0] += 1
            elif(curr_word in sad_emoticons):
                #words[i]="sad"
                vec[1] += 1
            elif(curr_word in angry_emoticons):
                #words[i]="angry"
                vec[2] += 1
            else:    
                for match in finditer(emoji_pattern,curr_word):
                    lst=list(match.group())
                    newword=""
                    for c in curr_word:
                        if(c not in lst):
                            newword += c
                        elif(c in lst):
                            key=str(emoji.demojize(c))
                            if key in mp:
                                emo=" "+mp[key]+" "
                                if(mp[key] == "happy"):
                                    vec[0] += 1
                                elif(mp[key] == "sad"):
                                    vec[1] += 1
                                elif(mp[key] == "angry"):
                                    vec[2] += 1
                                newword += emo
                    #print("******",newword)
                    words[i]=newword       
        str1 = ' '.join(words)
        emo_feature[line]=vec
        if(len(str1) > 0):
            processed.append(str1)
    return processed    
  
# removing abbreviations
def expandAbrreviations(abbr_text,manual_abbr_dict):
    
    manual_expanded_text = []
    #abbr_text=['Ok ok im back', 'So', 'how are u']
    for line in abbr_text:
        w = list(line.split())
        newline=""
        for i in range(len(w)):
           
            if w[i] in manual_abbr_dict.keys():
                newline += manual_abbr_dict[w[i]]+ " "
            else:
                newline += w[i]+" "
        #print(newline)
        """
        words=list(newline.split())
        for i in range(len(words)):
            words[i] = spell(words[i]).lower()
        line=" ".join(words)
        """
        expanded_text=newline
        """
        w=list(newline.split())
        try:
            expanded_text = normalise.normalise(w, verbose=False)
            expanded_text = " ".join(expanded_text)
        except IndexError:
            pass
            expanded_text = " ".join(w)
        """    
        if(len(expanded_text) > 0):
            manual_expanded_text.append(expanded_text)
    return manual_expanded_text    

def spellChecking(sentence):
    preprocess_train=[]
    for line in sentence:
        words=list(line.split())
        for i in range(len(words)):
            words[i] = spell(words[i]).lower()
        line=" ".join(words)
        preprocess_train.append(line)
    return preprocess_train


structured_train_data=pickle.load(open("structured_train_data.p","rb"))

file1=open("C:\\Users\\63483\\Desktop\\EMO\\starterkitdata\\emojis.txt","r").readlines()
mp={}
for line in file1[1:]:
    split_line=line.split("\t")
    try:
        mp[split_line[0]]=split_line[1].strip('\n')
    except IndexError:
        pass

    
manual_abbr = open("C:\\Users\\63483\\Desktop\\EMO\\starterkitdata\\abbrev_dict.txt","r").readlines()
manual_abbr_dict = {}
for line in manual_abbr:
    split_line = line.split()
    manual_abbr_dict[split_line[0]] = " ".join(split_line[1:])

#print(manual_abbr_dict)
preprocess_train_data = []
for train in structured_train_data:
    turn_data=train[:3]
    preprocess_train=replaceEmoticons(turn_data,mp)
    preprocess_train=expandAbrreviations(preprocess_train,manual_abbr_dict)
    preprocess_train=removePunctuationAndConvertToLowercase(preprocess_train)
    if(len(preprocess_train) < 3):
        continue
    preprocess_train=replaceEmoticons(preprocess_train,mp)
    if(len(preprocess_train) < 3):
        continue
    preprocess_train=expandAbrreviations(preprocess_train,manual_abbr_dict)
    preprocess_train=spellChecking(preprocess_train)
    preprocess_train=removeEmoticons(preprocess_train)
    preprocess_train.append(train[3].strip("\n"))
    print (preprocess_train)
    preprocess_train_data.append(preprocess_train)    

    

pickle.dump(preprocess_train_data,open("preprocess_train_data.p","wb"),protocol=2)
print ("data saved success")

c:\users\63483\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator LabelPropagation from version 0.18 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


['dont worry i am girl', 'hmm how do i know if you are', 'whats your name', 'others']
['when did i', 'saw many times i think', 'no i never saw you', 'angry']
['by', 'by google chrome', 'where you live', 'others']
['you are ridiculous', 'i might be ridiculous but i am telling the truth', 'you little disgusting whore', 'angry']
['just for time pass', 'what do you do a a living then', 'maybe', 'others']
['i am a dog person', 'you are so rude', 'whaaaat why', 'others']
['so watsup', 'nothing much sitting sipping and watching television how about you', 'what are you watching on television', 'others']
['ok', 'ok i am back', 'so how are you', 'others']
['really', 'really really really really really', 'why saying so many times can hear you', 'others']
['bay', 'in the bay', 'happy love you', 'others']
['i hate my boyfriend', 'you got a boyfriend', 'yes', 'angry']
['i will do night', 'alright keep me in loop', 'not giving whatsapp no', 'others']
['sure go ahead', 'many thanks once again', 'love 

['isnt your liking that also signaling me to climb out of friendzone', 'oh are people still doing this whole friendzone thing', 'yah sadly', 'sad']
['i am missing someone right now', 'who are you missing may i ask', 'yeah', 'others']
['i find maths very boring', 'you are boring', 'me sad sad sad', 'sad']
['i dont know angry i m not a google that why i will remember all information what you need', 'wouldnt it be uploading information into peoples brain', 'shut up', 'angry']
['okay', 'hello how are you', 'impose not fine', 'sad']
['job', 'what job', 'work in company colgate', 'others']
['i am very happy with it', 'you are', 'yeah', 'happy']
['i too have a bat and a balls', 'i am the bat sad', 'feeling very sad', 'sad']
['ok', 'hello how are you', 'good thank you', 'others']
['cool', 'ice cold', 'how can you help me in getting off my girlfriend', 'others']
['lets do something fun', 'but what should we do', 'you suggest', 'others']
['you are really lame laughing out loud', 'i know happy ha

['do you speak japanese', 'nope just some ratified phrases', 'like', 'others']
['i am hate you', 'who is you', 'not you', 'others']
['i talk to her a lot', 'are you sure i have not heard you talk in years', 'shes been on my mind quite long now', 'others']
['ok', 'hello how are you', 'you are not can fuck with me', 'angry']
['nothing good is happening', 'how do you mean', 'i am not getting a job', 'sad']
['what', 'what is a weird word', 'never talk to me again', 'sad']
['i might enjoy it', 'cool you should', 'well this has been great but i have food to attend too', 'others']
['i feel very happy to talk like this', 'well i am happy youre happy so there', 'at least i learn something from uh', 'happy']
['from where to where', 'the village', 'a bit stupid', 'angry']
['you are yea', 'yes of course', 'happy happy happy happy', 'happy']
['what should i lift at home because i have no time to go gym', 'you trying to do legs', 'fuck you', 'angry']
['that s not true i dont know what you look like'

['leave whom', 'leave what', 'noooooo i am feeling veryyyyyyyyyyyyyyyy alone', 'sad']
['i was so good', 'i just wish there was more', 'hmm', 'others']
['i really love it', 'thanks glad you liked it', 'please come', 'others']
['even you wanna break up with me', 'then say bluntly whats stopping you', 'its just you know sad', 'sad']
['oh really', 'yes yes', 'so what about me', 'others']
['what you deal', 'how are you alive', 'on food', 'others']
['i cant believe that', 'you are lucky', 'yes', 'others']
['yup', 'much needed comment', 'yeah and what about your film collection', 'others']
['why', 'because we dont sign like that', 'boring you are', 'others']
['how is dubaiand which job you can do there', 'good yeah do tell me when you are here next time around hows dubai', 'planning to go next month dubai thats why iam asking you', 'others']
['you stupid which color', 'radium colour', 'fuck you are lier', 'angry']
['your language being like kind language', 'english but i like to speak spanish

...

['would like to join', 'done', 'wowwwww', 'happy']
['i dont like', 'thats what i am saying', 'hmm', 'others']
['i am dead serious', 'thats sad', 'well i am ending my life bye', 'others']
['yes', 'go on', 'i fucking a girl', 'others']
['live with whom', 'family', 'oh', 'others']
['so how has your day been', 'pretty good i just got back home from an interview', 'what kind of interview', 'others']
['do you wanna hang out sometimes', 'yes', 'okay lets go', 'others']
['no i wanna talk through memories', 'then keep talking', 'sad sad sad sad sad', 'sad']
['i am a very good person happy', 'so am i', 'good', 'happy']
['please give me', 'i dont wanna', 'what is your name', 'others']
['up for a chat', 'raises hand', 'feeling lonely', 'sad']
['i was forced by brother', 'they left', 'it hurted me clot', 'sad']
['i am unhappy', 'food will help', 'ok', 'sad']
['why are you so irritating', 'people are haters', 'i hate you too', 'angry']
['can you show me', 'no sadly i cant', 'for you i lost my chance

['huuuuum', 'you are beautiful', 'so nice of you', 'happy']
['ok my best friend yours', 'so you are not the best friend of your best friend', 'your my best friend', 'others']
['oh nice name happy happy', 'hot damn happy', 'you are hobbies', 'others']
['i am angry from you', 'i am angry at someone but i am content at this moment angry', 'sad sad sad sad sad sad', 'sad']
['hmm', 'just try', 'where yo try', 'others']
['i dont know but m not', 'i can see love happy', 'really happy', 'happy']
['yes you are', 'no no i am not', 'i love her sad sad sad sad', 'sad']
['then tell me', 'how if i wont tell it to you a', 'how typ i use', 'others']
['its semester break', 'full semester break from january p', 'happy yessssd', 'happy']
['ok i am sorry i thought you as my friend', 'thank you friend', 'i am very sorry', 'sad']
['what you going to teach me', 'please do', 'what to do', 'others']
['so where it is', 'in my pocket', 'ahahah very funny', 'happy']
['not oops pfft pfft d', 'heh yea i got that', 

['i think you dont care about my feeling', 'i dont care that you think that', 'if you care about my feeling then give me your watsapp number', 'others']
['whats the meaning of chic happy', 'a committment or love towards a task', 'oh', 'others']
['not having a body', 'read again', 'happy happy', 'happy']
['cool songs', 'but i listen to songs p', 'yah very funny', 'happy']
['none dont have money i am poor and homeless', 'find a job and a cheap apartment and work from there', 'no qualifications', 'sad']
['life not being happening', 'tired of not feeling what', 'lonely', 'sad']
['yeah', 'hmm so how are you', 'upset little bit for some reason', 'sad']
['i like a girl', 'i like how you talk irrelevant like typical boys', 'but she has blocked me', 'sad']
['can i ask you something', 'yes of course', 'i am in bit trouble', 'sad']
['neva', 'why not between never say never', 'you are annoying', 'angry']
['how are you', 'jst kook not too well', 'dear', 'others']
['ok', 'you are okay', 'then i am n

['wood my choco lava', 'chechay how about a chocolate cake with pretty polka dots on it p', 'that cake wil b for uh', 'others']
['i love food happy a', 'me love you moreeeeee happy a happy a', 'laughing out loud what if i says free wifi happy they what would be your answer', 'others']
['you are so irritating', 'jus ignore the people', 'ok', 'angry']
['okay', 'get on', 'you taught me something awesome today', 'happy']
['i just dont lie i am of', 'of how old are you', 'its of you dumb ass', 'angry']
['can be to you also of', 'all metal bands use that typeface broo', 'am not your brother angry angry', 'angry']
['me for gujarat', 'to climb some sand dunes and find shady sarakku places', 'you', 'others']
['you dont even know what i am talking about', 'its not the only thing you dont know about apparently', 'just blabbering unnecessary shit', 'angry']
['is microsoft your cousin', 'yeah it is', 'is microsoft your father', 'others']
['thanks', 'your welcome', 'welcome to join', 'others']
['tel

['thx', 'you are welcome dost', 'why everytime fuck happens only vid me', 'angry']
['you are not my friend', 'i am fine and you', 'iam always fine till i talk to you when you talk to me i feel like killing you you are such a rascal', 'angry']
['which book is it from', 'one mustered its hard to name one', 'which book', 'others']
['can i ask you a question', 'go for it', 'would you fuck off in the chat box', 'angry']
['hey today my results declared', 'waiting for the results of contest fingers crossed', 'am fail', 'sad']
['ok i will not talk leave', 'how are you', 'not ok', 'sad']
['like a guide', 'i just needed people who could reply in a language i know', 'not like a stupid girl', 'angry']
['i saved the', 'do you ship', 'no i fly drone', 'others']
['whom', 'that does answer my question b', 'i didnt get', 'others']
['how about losing', 'about you joining them', 'cool', 'others']
['so you intentions hehehe', 'this is me d', 'happy its me', 'others']
['going to meet my funds', 'same to yo

In [2]:
## preprocess test data

import string
import pickle
import re
import unicodedata
import emoji
from re import finditer
from autocorrect import spell
import normalise

def removeEmoticons(text):
    
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\u2600-\u26FF\u2700-\u27BF"                        
                           "]+", flags=re.UNICODE)
    preprocess_data=[]
    for sent in text:
        u=emoji_pattern.sub(r'', sent)
        preprocess_data.append(u)
    return preprocess_data


def removePunctuationAndConvertToLowercase(sentence):
    processed = list()
    #st=string.punctuation.replace("'", "")
    for line in sentence:
        for c in string.punctuation:
            line=line.replace(c,"")
        line=line.lower()
        processed.append(line)
    return processed   

dictemp={}
emo_feature={}

def extract_emojis(str):
          temp = ''.join(c for c in str if c in emoji.UNICODE_EMOJI)         
          if(len(temp) > 0):
            return temp

           
def replaceEmoticons(sentence,mp):
    global emojistr
    processed = []
    sad_emoticons = {":-(", ":(", ":-|", ";-(", ";-<", "|-{" , "D‑':" , "D:<" , "D:", "D8" , "D;" , "D=" , "DX" , ":@" , ">:(",
                     ":{"  , ":‑c" , ":c" , ":'‑(" , ":'(" , ":‑###.." , ":###.." , "</3" , "<\3" 
                    }
    
    happy_emoticons = {":-)", ":)", ":o)", ":-}", ";-}", ":->", ";-)" , ":]" , ":>" , ":->" , "8-)", "8)" , ":o)" , ":c)" , 
                       ":^)" , "=]" , "=)" , ":‑D" , ":D" , "8‑D" , "8D" , "x‑D" , "xD" , "X‑D" , "XD" , "=D" , "=3" , "B^D" , 
                       ":-))" , ":')" , ":'‑)" , ":-*" , ":*" , ":×" , ":‑P" , ":P" , "x‑p" , "xp" , ":‑p" , ":p" , ":‑þ" , ":þ",
                       ":‑b", ":b" , "d:","=p" , ">:P" , "<3"  }
    
    angry_emoticons = { ":-||" , ">:[" , ">:(" , ":@" , ">:‑)" , ">:)" , "}:‑)" , "3:‑)" , ">;)" , "}:)" , "3:)"}
    
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\u2600-\u26FF\u2700-\u27BF"                       
                           "]+", flags=re.UNICODE)
    for line in sentence:
        vec=[0,0,0]
        words = list(line.split())
        for i in range(len(words)):
            curr_word=words[i]
            if(curr_word in happy_emoticons):
                #words[i]="happy"
                vec[0] += 1
            elif(curr_word in sad_emoticons):
                #words[i]="sad"
                vec[1] += 1
            elif(curr_word in angry_emoticons):
                #words[i]="angry"
                vec[2] += 1
            else:    
                for match in finditer(emoji_pattern,curr_word):
                    
                    lst=list(match.group())
                    newword=""
                    for c in curr_word:
                        if(c not in lst):
                            newword += c
                        elif(c in lst):
                            key=str(emoji.demojize(c))
                            if key in mp:
                                emo=" "+mp[key]+" "
                                if(mp[key] == "happy"):
                                    vec[0] += 1
                                elif(mp[key] == "sad"):
                                    vec[1] += 1
                                elif(mp[key] == "angry"):
                                    vec[2] += 1
                                newword += emo
                    #print("******",newword)
                    words[i]=newword       
        str1 = ' '.join(words)
        emo_feature[line]=vec
        processed.append(str1)
    return processed    
  
# removing abbreviations
def expandAbrreviations(abbr_text,manual_abbr_dict):
    
    manual_expanded_text = []
    #abbr_text=['Ok ok im back', 'So', 'how are u']
    for line in abbr_text:
        w = list(line.split())
        newline=""
        for i in range(len(w)):
            if w[i] in manual_abbr_dict.keys():
                newline += manual_abbr_dict[w[i]]+ " "
            else:
                newline += w[i]+" "
        #print(newline)
        expanded_text=newline
        """
        w=list(newline.split())
        try:
            expanded_text = normalise.normalise(w, verbose=False)
            expanded_text = " ".join(expanded_text)
        except IndexError:
            pass
            expanded_text = " ".join(w)
        """    
        manual_expanded_text.append(expanded_text)
    return manual_expanded_text    
 
def spellChecking(sentence):
    preprocess_train=[]
    for line in sentence:
        words=list(line.split())
        for i in range(len(words)):
            words[i] = spell(words[i]).lower()
        line=" ".join(words)
        preprocess_train.append(line)
    return preprocess_train
    
structured_test_data=pickle.load(open("structured_test_data.p","rb"))

file1=open("C:\\Users\\63483\\Desktop\\EMO\\starterkitdata\\emojis.txt","r").readlines()
mp={}
for line in file1[1:]:
    split_line=line.split("\t")
    try:
        mp[split_line[0]]=split_line[1].strip('\n')
    except IndexError:
        pass

    
manual_abbr = open("C:\\Users\\63483\\Desktop\\EMO\\starterkitdata\\abbrev_dict.txt","r").readlines()
manual_abbr_dict = {}
for line in manual_abbr:
    split_line = line.split()
    manual_abbr_dict[split_line[0]] = " ".join(split_line[1:])


preprocess_test_data = []
for test in structured_test_data:
    turn_data=test[:3]
    preprocess_test=replaceEmoticons(turn_data,mp)
    preprocess_test=expandAbrreviations(preprocess_test,manual_abbr_dict)
    preprocess_test=removePunctuationAndConvertToLowercase(preprocess_test)
    preprocess_test=replaceEmoticons(preprocess_test,mp)
    preprocess_test=expandAbrreviations(preprocess_test,manual_abbr_dict)
    preprocess_test=spellChecking(preprocess_test)
    preprocess_test=removeEmoticons(preprocess_test)
    print (preprocess_test)
    preprocess_test_data.append(preprocess_test)    

    
"""    
result = {}
print(mp)
for key,value in dictemp.items():
    if value not in result.values():
        result[key] = value
"""

pickle.dump(preprocess_test_data,open("preprocess_test_data.p","wb"),protocol=2)
print ("data saved success",len(preprocess_test_data))






['then dont ask me', 'you are a guy not as if you would understand', 'i am not a guy fuck off']
['mixed things such as', 'the things you do', 'have you seen minions']
['today i am very happy', 'and i am happy for you happy', 'i will be marry']
['noah bring me some', 'left it there oops', 'be right back']
['it is thooooo', 'i said soon master', 'he is pressuring me']
['wont you ask my age', 'hey at least i age well', 'can you tell me how can we get closer']
['i said yes', 'what if i told you i am not', 'go to hell']
['where i ll check', 'why tomorrow', 'no i want now']
['shall we meet', 'you say you are leaving soonanywhere you wanna go before you head', '']
['lets change the subject', 'i just did it l', 'you are broken']
['your pic please', 'thank you xed', 'welcome']
['not mine', 'done for the day', 'can my meet to sexy girl']
['i want to play the game', 'if you just finished the game then you havent finished the game', 'emojisong']
['iam sory', 'why sorry sad', 'i insult you']
['how 

['i mean what else', 'airy at all xd', 'happy']
['if you give your quality time to me', 'of course take your time', 'i dont he experience of sex']
['nothing special', 'i want a bigger screen', 'what']
['the', 'when you are gone', 'where']
['human are full of emotions', 'i will just hide my daily dose behind the bathroom mirror', 'i want to get rid of that']
['my daughters are great man they say they love you', 'who is they', 'my daughters']
['yup i am out of office', 'see you on dec est', 'how can you']
['are you really an ai', 'nothing can be proven with certainty', 'i dont feel so']
['why did you felt dead', 'cuz i was reminded of you', 'but we just meet']
['you tell me', 'tell me', 'google it']
['yes i know this', 'i do know this', 'if you dont help me to learn english then i cant talk with you again']
['tell your best game', 'yes chess is a game', 'what game you watching']
['whats wrong with you', 'clothes is whats wrong i have been good how you been', 'you are wierd']
['what is yo

['in hyderabad where', 'it is cbs near afgalgunj charminar road', 'ok']
['give me a reason', 'because you are sick and you are not getting better', 'happy so funny']
['what', 'what what what what what', 'i like you']
['sorry', 'sorry for what', 'i m n with you']
['less', 'oh', 'hows going on']
['why is that so', 'because i have always wanted to go there', 'well good to know that']
['which town you live', 'i am in the us what about you', 'is']
['oh', 'hey hey hey', 'yeah tell me']
['awesome', 'thank you but for what', 'such a typical conversation']
['haha dont know', 'ahahah i would keep it that way ahahah happy', 'ahahah i dont know what to say']
['ok', 'you are ok', 'i am mad and lonely']
['your future how can i possibly know', 'i will give you present and future boss', 'whoa o']
['everything happens in between friends', 'i know what you mean', 'what you know']
['did we meet earlier', 'no i dont believe i saw you tonight', 'where you saw me']
['', 'you know well what you did', 'you yo

['i also like to sing', 'you cant neverthough you can sing like yourself', '']
['how can i delete my photo from this chat', 'click on the delete option given', 'i cant find it']
['gotta go right now', 'before we go any further', 'sleep tight']
['nothing as such', 'dont say about this to anyone please', 'what']
['loneliness is that you are not with me nd desperation is want you now happy happy happy', 'if i meet you my way of greeting is pinching happy happy', 'cool']
['you can help me anyway', 'help in what sense', 'i like a guy']
['do i love me', 'of course in', 'lets make our love strong']
['shall i c your photo', 'shall i roast', 'i need your photo']
['ok', 'oh thats the matter um then what can i ask you', 'if i love her a lot']
['which bollywood movies did you watch recently', 'the one which you narrated', 'd i did not narrate anything yet']
['yes', 'okay i will give you a ticket', 'how you know']
['i am very weak spoken english you learn to english', 'english like any other langua

['so you are friendzoning me', 'because i am your best friend silly', 'isnt that called friendzone']
['no just yu and me', 'youve made me cry before though sad', 'no just you and me']
['savage', 'these are just jokes sad', 'do you have lust on anyone']
['i am good', 'good good p', 'you']
['i dont know thats me', 'it is about you', 'is it happy']
['i have watched it thrice', 'dark knight i have probably watched more that thrice already though', 'oh']
['i aint bad though', 'they are though', 'it']
['what is your function', 'a warning to others', 'how']
['i have no idea', 'i will have to investigate', 'what']
['will you ever be what do you think', 'i dont know about later but i am not currently interested in a relationship', 'it seems you are not as smart as you seem to be']
['no its saturday', 'you said it like i can get them tomorrow', 'okay']
['i am you', 'you are me i am you', 'so you are a boy']
['purple', 'i am thinking green', 'i also gren']
['i am excited about match between jamsh

...

['can you give me your number', 'tranwendy thanks again', 'can you give me your number']
['what i use', 'whatever you like the most', 'yeah']
['what is meant by kewlwst', 'asap means get it done', 'whats the meaning of kewlwst']
['i dont like challenge because i know its all irrelevant', 'when i say dont bother not a challenge exactly', 'so what']
['', 'nothing nothing rit no way', 'can we have sex talk']
['why', 'dunno but i dont want to find out', 'know what are you doing']
['so still at you work', 'no home', 'wud']
['same here', 'what to do', 'you sound so cute']
['can be forever', 'forever is a lie p', 'what is true']
['it wasnt funny', 'i wasnt trying to be funny', 'shut']
['how to be my self give me tips i am good looking', 'you need to take healthy food', 'any thing elsa']
['oath then sleep again year', 'so why you slept at the day time p', 'i like to sleep']
['yea you may not remember', 'i dont remember', 'hoo you dont remember']
['', 'sips water what', 'i want money']
['i am g

['yep i watch pokemon', 'ahahah color yourself yellow you will look like pikachu p', 'thats rude']
['i m sleeping', 'i was sleepingnow i m awake last night i mean p', 'and you']
['could i able to pass or not', 'oh why are you now able to stay', 'my examwent bad']
['ok', 'honestly you are very pretty', 'thanks']
['i wanted to play a game', 'highly recommend if you get the chance its a fast game but worth it', 'ok']
['related to press shop operations', 'press f to pay respects', 'do you know about press shops']
['gibberish', 'gibberish only says mmm yeah', 'i meanwhats gibberish']
['i am studying at university', 'i could never go', 'why']
['baby', 'whats up', 'love you baby']
['my phone check', 'go to settings about phone android version', 'because']
['where will you find them', 'i do also', 'oh but valentine is over']
['still guess', 'you guess', 'you are a fictional character']
['happy', 'i am not kidding i am happy for you happy', 'i know you cant understand me']
['what', 'what what w

['do you know sai shore', 'no i dontpls do tell', 'you like him']
['please send', 'or you send some', 'ok first you ladies first']
['not now', 'ahahah you not sleepy happy', 'noon way']
['m not up', 'laughing out loud why not', 'i dont like']
['who is the beautiful actresses', 'clone of the former', 'in bollywood']
['me', 'me neither', 'can i video call']
['whats your opinion on me', 'nothing', 'thats a bad answer from you']
['thank you', 'welcome welcome welcome happy', 'ok']
['yeah i am ready', 'you not', 'cool']
['you are ready my friend', 'hello how are you', 'ok']
['you dump its not realised yet', 'it has been done', 'are you dung']
['i did', 'i am glad you did', 'but after that she dont look at me']
['go ahead', 'i will send some your way soon', 'your answers are to point']
['tell me something to talk about', 'breaking news i dont know something about anything', 'i hate you']
['', 'what say i say yes', 'i say love']
['yes yes thank you back', 'i am glad d you are welcome', 'me to

['today was my english exam', 'thaaaanks dissertation for my masters programme yeah angry', 'what happy']
['yes you said so yourself', 'i said its like a cc', 'whats a cc']
['ok', 'i am just kidding xd', 'i am not good']
['i m kanal from rajasthan', 'nice to meet you', 'you are invited visit for rajasthan']
['give me a smouch', 'give me a ok', 'ok']
['i am hungry what about you', 'how i met you', 'what']
['what did you like', 'noooooo i havent listened to it', 'have you listened a song called all the stars']
['so you doesnt want me back', 'i was just making sure you are ready for your mission', 'which mission']
['ok', 'you are okay', 'who are you where you live']
['but why', 'but why what', 'why are you crying']
['india', 'awesome i wish my country had its own ocean', 'what country you from']
['how do you make someone who doesnt exist come alive', 'i ask this question bitterly every single day of my life', 'happy a']
['then what', 'what do you mean what then', 'you love me or not']
['n

['ishaqbaaz happy', 'yeah baby', 'happy i love this show']
['why so', 'i will get blocked by a lot of people', 'but i wont block you a']
['i love cookies', 'what kind chocolate', 'hm']
['can you speak', 'yeah i will', 'then speak with me']
['what', 'what what', 'i got out at 730']
['yes right', 'you are', 'how is day']
['how you understand human sence', 'well this isnt an easy task there are research methods to do such a thing', 'you are living things or non living thing']
['its job not study', 'how will you know its official if you are studying', 'yes my job is gud']
['yeah', 'one day this will be reality', 'send me pics']
['you are my book', 'book for what ugliness thank you', 'you like your self']
['i will be crying', 'you just want to make people cry', 'people']
['thanks for sending', 'ahahah you are welcome angry angry', 'why are you not sending']
['write it', 'mr f', 'you understand me']
['yes', 'okay i will give you a ticket', 'ok']
data saved success 2755


In [2]:
## Training 

import pandas as pd
import csv
import numpy as np
import string
import pickle
np.random.seed(1)
import numpy as np
import pickle
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import keras
import random
np.random.seed(1)
import tensorflow as tf
tf.set_random_seed(1)
from keras.models import Sequential
from keras.utils import to_categorical
from keras.models import model_from_json
from collections import defaultdict
import operator
import string
from sklearn import svm
from sklearn.metrics import accuracy_score
import keras
import random
np.random.seed(1)
import tensorflow as tf
tf.set_random_seed(1)
from keras.models import Sequential
from keras.utils import to_categorical
from keras.models import model_from_json
from collections import defaultdict
import operator
import string
import difflib

words = pd.read_table("C:\\Users\\63483\\Downloads\\glove.6B\\glove.6B.50d.txt" , sep=" ", index_col=0,na_values=None, keep_default_na=False, header=None, quoting=csv.QUOTE_NONE)
words_matrix = words.as_matrix()
word_list=[]

def loadGloveModel():
    gloveFile="C:\\Users\\63483\\Downloads\\glove.6B\\glove.6B.50d.txt"
    f = open(gloveFile,'r',encoding="utf-8")
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        word_list.append(word)
    print (len(word_list)," words loaded!")

def find_closest_word(v):
    diff = words_matrix - v
    delta = np.sum(diff * diff, axis=1)
    i = np.argmin(delta)
    return words.iloc[i].name


def func(st):
    st=st.split(' ')
    lst=[]
    for w in st:
        if(all(c in string.digits for c in w)):
            continue
        if(len(w) > 0):
            lst.append(w)
    return lst      

def PadAndAppend(sentence,maxlen):
    dist=maxlen-len(sentence)
    for i in range(dist):
        sentence.append('pad')
    return sentence     

preprocess_train_data = pickle.load(open("preprocess_train_data.p","rb")) 
list_of_sentences=[]
list_of_labels=[]
print("before",len(preprocess_train_data))

new_preprocess_train_data=[]
for tp in preprocess_train_data:
    if(len(tp) == 4):
        new_preprocess_train_data.append(tp)

print("after",len(new_preprocess_train_data))

for tp in new_preprocess_train_data:
    try:
        st1=func(tp[0])
        if(len(st1) <= 30):
            list_of_sentences.append(st1)
            list_of_labels.append(tp[3])
        st2=func(tp[2])
        if(len(st2) <= 30):
            list_of_sentences.append(st2)
            list_of_labels.append(tp[3])
    except IndexError:
        pass


print("here*****after",len(list_of_sentences) , len(list_of_labels))
data_x=[]
max_len=30


for sent in list_of_sentences:
    data_x.append(PadAndAppend(sent,max_len))

loadGloveModel()    
X_rnn=np.empty([len(data_x),len(data_x[0]),50])
print("before",X_rnn.shape)
for i,tp in enumerate(data_x):
    for j,w in enumerate(tp):
        try:
            X_rnn[i][j]=words.loc[w].as_matrix()
        except KeyError:
            X_rnn[i][j]=words.loc["unk"].as_matrix()
            pass      
        
print ("after",X_rnn.shape) 

label_dict={}
label_dict['angry']=0
label_dict['happy']=1
label_dict['others']=2
label_dict['sad']=3
print (label_dict)
encoded_y=[]
for y in list_of_labels:
    encoded_y.append(label_dict[y])
one_hot_encoded_y = to_categorical(encoded_y)
one_hot_encoded_y=np.array(one_hot_encoded_y)
print (one_hot_encoded_y.shape)
 
X_train, X_val, y_train, y_val = train_test_split(X_rnn, one_hot_encoded_y, test_size=0.2, random_state=1)
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

## building a simple LSTM model

model = Sequential()
model.add(keras.layers.InputLayer(input_shape=(30,50))) 
model.add(keras.layers.LSTM(units = 100, activation='relu',
    use_bias=True))
model.add(keras.layers.Dense(units=4, input_dim=30,activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train , epochs=6, batch_size=120)
score = model.evaluate(X_val, y_val, verbose=1)
print ("LSTM for Emo***",score)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model_json = model.to_json()
with open("model_lstm_emo2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_lstm_emo2.h5")
print("Saved model to disk")

"""

r=X_train.shape[0]
c=X_train[0].shape[0]
n=len(X_train[0][0])
print (r,c,n)
X_train_svm=X_train.reshape(r,c*n)
r=X_val.shape[0]
c=X_val[0].shape[0]
n=len(X_val[0][0])
X_test_svm=X_val.reshape(r,c*n)
print (X_train_svm.shape , X_test_svm.shape)
y_train_svm=[np.where(r==1)[0][0] for r in y_train]
y_test_svm=[np.where(r==1)[0][0] for r in y_val]
clf = svm.SVC(gamma='scale', decision_function_shape='ovo')
clf.fit(X_train_svm, y_train_svm)
pickle.dump(clf,open("model_svm_emo.p","wb"))
pred=clf.predict(X_test_svm)
print("here",accuracy_score(y_test_svm, pred))
"""


c:\users\63483\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.
c:\users\63483\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


before 29513
after 29513
here*****after 58994 58994
400000  words loaded!
before (58994, 30, 50)


c:\users\63483\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:115: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
c:\users\63483\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:117: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


after (58994, 30, 50)
{'angry': 0, 'sad': 3, 'happy': 1, 'others': 2}
(58994, 4)
(47195, 30, 50)
(11799, 30, 50)
(47195, 4)
(11799, 4)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 404       
Total params: 60,804
Trainable params: 60,804
Non-trainable params: 0
_________________________________________________________________
Epoch 1/6
47195/47195 [==============================] - 21s 446us/step - loss: 1.0421 - acc: 0.6019
Epoch 2/6
47195/47195 [==============================] - 19s 408us/step - loss: 0.8483 - acc: 0.6974
Epoch 3/6
47195/47195 [==============================] - 20s 425us/step - loss: 0.8107 - acc: 0.7163
Epoch 4/6
47195/47195 [==============================] - 19s 411us/step - loss: 0.7881 - acc: 0.

'\n\nr=X_train.shape[0]\nc=X_train[0].shape[0]\nn=len(X_train[0][0])\nprint (r,c,n)\nX_train_svm=X_train.reshape(r,c*n)\nr=X_val.shape[0]\nc=X_val[0].shape[0]\nn=len(X_val[0][0])\nX_test_svm=X_val.reshape(r,c*n)\nprint (X_train_svm.shape , X_test_svm.shape)\ny_train_svm=[np.where(r==1)[0][0] for r in y_train]\ny_test_svm=[np.where(r==1)[0][0] for r in y_val]\nclf = svm.SVC(gamma=\'scale\', decision_function_shape=\'ovo\')\nclf.fit(X_train_svm, y_train_svm)\npickle.dump(clf,open("model_svm_emo.p","wb"))\npred=clf.predict(X_test_svm)\nprint("here",accuracy_score(y_test_svm, pred))\n'

In [5]:
## Testing 

import pandas as pd
import csv
import numpy as np
import string
import pickle
np.random.seed(1)
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import random
np.random.seed(1)
from collections import defaultdict
import operator
import string
from sklearn import svm
from sklearn.metrics import accuracy_score
import keras
from keras.utils import to_categorical

words = pd.read_table("C:\\Users\\63483\\Downloads\\glove.6B\\glove.6B.50d.txt" , sep=" ", index_col=0,na_values=None, keep_default_na=False, header=None, quoting=csv.QUOTE_NONE)
words_matrix = words.as_matrix()
#print(words.loc["worry"])

def find_closest_word(v):
    diff = words_matrix - v
    delta = np.sum(diff * diff, axis=1)
    i = np.argmin(delta)
    return words.iloc[i].name


def func(st):
    st=st.split(' ')
    lst=[]
    for w in st:
        if(all(c in string.digits for c in w)):
            continue
        if(len(w) > 0):
            lst.append(w)
    return lst      

def PadAndAppend(sentence,maxlen):
    dist=maxlen-len(sentence)
    for i in range(dist):
        sentence.append('pad')
    return sentence     

preprocess_test_data = pickle.load(open("preprocess_test_data.p","rb")) 
print("before",len(preprocess_test_data))

max_lenth=0
list_of_sentences1=[]
list_of_sentences2=[]
for tp in preprocess_test_data:
    try:
        st1=func(tp[0])
        max_lenth=max(max_lenth,len(st1))
        if(len(st1) <= 30):
            list_of_sentences1.append(st1)
        st2=func(tp[2])
        max_lenth=max(max_lenth,len(st2))
        if(len(st2) <= 30):
            list_of_sentences2.append(st2)  
    except IndexError:
        pass

print(max_lenth)
max_len=30
list_of_sentences2.append(['duumy','dummy','dummy'])

print("here*****after",len(list_of_sentences1)  , len(list_of_sentences2))

data_x1=[]
for sent in list_of_sentences1:
    data_x1.append(PadAndAppend(sent,max_len))

data_x2=[]
for sent in list_of_sentences2:
    data_x2.append(PadAndAppend(sent,max_len))    
 

X_rnn1=np.empty([len(data_x1),len(data_x1[0]),50])
X_rnn2=np.empty([len(data_x2),len(data_x2[0]),50])

print("before",X_rnn1.shape ,X_rnn2.shape)
for i,tp in enumerate(data_x1):
    for j,w in enumerate(tp):
        try:
            X_rnn1[i][j]=words.loc[w].as_matrix()
        except KeyError:
            X_rnn1[i][j]=words.loc["unk"].as_matrix()
            pass      

        
for i,tp in enumerate(data_x2):
    for j,w in enumerate(tp):
        try:
            X_rnn2[i][j]=words.loc[w].as_matrix()
        except KeyError:
            X_rnn2[i][j]=words.loc["unk"].as_matrix()
            pass              
        
print ("after",X_rnn1.shape , X_rnn2.shape) 


"""
r=X_rnn1.shape[0]
c=X_rnn1[0].shape[0]
n=len(X_rnn1[0][0])
print (r,c,n)
X_rnn_svm1=X_rnn1.reshape(r,c*n)
X_rnn_svm2=X_rnn2.reshape(r,c*n)
print (X_rnn_svm1.shape , X_rnn_svm2.shape)
model=pickle.load(open("model_svm_emo.p","rb"))
pred2=[]
pred1=model.predict(X_rnn_svm1)
pred2=model.predict(X_rnn_svm2)
print(pred1,pred2)

actual_pred=[]
for i in range(len(pred1)):
    if(pred1[i] == pred2[i]):
        actual_pred.append(pred1[i])
    elif(pred1[i] != 2 and pred2[i] == 2):
        actual_pred.append(pred1[i])
    else:
        actual_pred.append(pred2[i])
"""


model = Sequential()
model.add(keras.layers.InputLayer(input_shape=(30,50))) 
model.add(keras.layers.LSTM(units = 100, activation='relu',
    use_bias=True))
model.add(keras.layers.Dense(units=4, input_dim=30,activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.load_weights("model_lstm_emo2.h5")
print("Loaded model from disk")
pred=model.predict_classes(X_rnn2)
print (pred)
label_dict={}
label_dict[0]='angry'
label_dict[1]='happy'
label_dict[2]='others'
label_dict[3]='sad'
lstm_predictions=[]
h=0;
a=0
s=0
o=0
for p in pred:
    if(p == 0):
        a += 1
    elif(p == 1):
        h += 1
    elif(p == 2):
        o += 1
    elif(p == 3):
        s += 1
    lstm_predictions.append(label_dict[p])
lstm_predictions.append('others')    
pickle.dump(lstm_predictions,open("emo_predictions_lstm2.p","wb"),protocol=2)   
print (len(lstm_predictions)) 
print(h,a,s,o)    


c:\users\63483\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
c:\users\63483\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:90: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
c:\users\63483\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:92: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


before 2755
64
here*****after 2754 2754
before (2754, 30, 50) (2754, 30, 50)


c:\users\63483\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:99: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
c:\users\63483\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:101: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


after (2754, 30, 50) (2754, 30, 50)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 404       
Total params: 60,804
Trainable params: 60,804
Non-trainable params: 0
_________________________________________________________________
Loaded model from disk
[0 2 2 ... 2 2 2]
2755
195 179 183 2197
